# Neural Networks Sprint Challenge

## 1) Define the following terms:

- Neuron
- Input Layer
- Hidden Layer
- Output Layer
- Activation
- Backpropagation

**Neuron:**
A node in A Neural Network. They have a structure analogous to that of biological neurons. Neurons read the activation state of a bunch of neurons in the previous layer (each weighed differently), and use that information to produce a single output value.


**Input Layer:**
The first layer of a NN. It's a series of variables that correspond directly to a feature in the data. In image recognition, for example, they could be the brightness of a particular pixel.

**Hidden Layer:**
Internal layers of the NN, connecting input and output layers. They are a series of functions, each of which takes in the values of all the variables in the previous layer (starting with the input layer) and produces a single number as output. The output value is generated as a sum of the values of all the nodes in the previous layer, each multiplied by a weight. That summation gets passed through a squishification function, and added to a bias to produce the output. Hidden layers don't need to correspond to any recognizable feature of the outside world.

**Output Layer:**
The final layer of the NN. Each node is a function like the hidden layers, but its output corresponds to the NN's predictions for a single outcome variable. Note that the nodes in this layer don't usually have an activation function.

**Activation Function:**
Each neuron must aggregate inputs and produce a single output. The activation function shapes that output to be within useful bounds. One can use several possible activation functions, but common functions (sigmoid, tanh) will map the whole numberline to a small range ((0,1) or (-1,1)) or get rid of negative numbers (ReLU). Note that all the nodes in a layer of the NN tend to have the same activation function.

**Backpropagation:** Short for "Backwards Propagation of errors" and refers to a specific algorithm for how weights in a neural network are updated in reverse order at the end of each training epoch. It 'assigns blame' to weights and works backwards to nudge the weights until they 'improve' the prediction

## 2) Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 1  | 1  | 1  | 1 |
| 1  | 0  | 1  | 0 |
| 0  | 1  | 1  | 0 |
| 0  | 0  | 1  | 0 |

In [1]:
import numpy as np
import pandas as pd

In [2]:
class Perceptron(object):

    def __init__(self, no_of_inputs, threshold=100, learning_rate=0.01):
        self.threshold = threshold
        self.learning_rate = learning_rate
        self.weights = np.zeros(no_of_inputs + 1)
           
    def predict(self, inputs):
        summation = np.dot(inputs, self.weights[1:]) + self.weights[0]
        if summation > 0:
            activation = 1
        else:
            activation = 0            
        return activation

    def train(self, training_inputs, labels):
        for _ in range(self.threshold):
            for inputs, label in zip(training_inputs, labels):
                prediction = self.predict(inputs)
                self.weights[1:] += self.learning_rate * (label - prediction) * inputs
                self.weights[0] += self.learning_rate * (label - prediction)

inputs = np.array([[1,1,1],
                   [1,0,1],
                   [0,1,1],
                   [0,0,1]])

correct_outputs = np.array([[1],
                            [0],
                            [0],
                            [0]])

pn = Perceptron(no_of_inputs=3, threshold=100, learning_rate=0.01)
pn.train(inputs, correct_outputs)

In [3]:
print("     AND Gate")
for row in inputs:
    print(f'{row[0]} {row[1]} {row[2]} --> {pn.predict(row)}')

     AND Gate
1 1 1 --> 1
1 0 1 --> 0
0 1 1 --> 0
0 0 1 --> 0


## 3) Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights. 
- Your network must have one hidden layer. 
- You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
- Train your model on the Heart Disease dataset from UCI:

[Github Dataset](https://github.com/ryanleeallred/datasets/blob/master/heart.csv)

[Raw File on Github](https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv)


### Imports

In [4]:
from mlxtend.data import mnist_data
import matplotlib.pyplot as plt
from mlxtend.preprocessing import shuffle_arrays_unison
from sklearn.preprocessing import StandardScaler

ModuleNotFoundError: No module named 'mlxtend'

### Load and Inspect Data

In [ ]:
url="https://raw.githubusercontent.com/ryanleeallred/datasets/master/heart.csv"
df=pd.read_csv(url)
print(df.columns)
print(df.shape)

df.head()

In [ ]:
y = df.target.values
X = df.drop(columns = ['target']).values

In [ ]:
X.shape

In [ ]:
y=y.reshape(-1,1)

In [ ]:
print(y.shape)
y[0]

### NN Model

In [ ]:
# inputs = 13
# 2 hidden layers of 16 each, 
# output of 1 variables (the probability prediction)
class NN_2L16(object):
    def __init__(self):
        self.inputs = 13
        self.L1Nodes = 16
        self.L2Nodes = 16
        self.outputNodes = 1

        # Initlize Weights
        self.L1_weights = np.random.randn(self.inputs, self.L1Nodes) # (784x16)
        self.L2_weights = np.random.randn(self.L1Nodes, self.L2Nodes) # (16x16)
        self.output_weights = np.random.randn(self.L2Nodes, self.outputNodes) # (16x10)

    def feed_forward(self, X):
        # Sum and activate flows to L1
        self.activated_L1 = self.sigmoid(np.dot(X, self.L1_weights)) 
        # Sum and activate flows to L2
        self.activated_L2 = self.sigmoid(np.dot(self.activated_L1, self.L2_weights))
        # Sum and activate flows to output
        self.activated_output = self.sigmoid(np.dot(self.activated_L2, self.output_weights))
        return self.activated_output
        
    def sigmoid(self, s):
        return 1/(1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    def backward(self, X, y, output):
        ## backward propgate through the network, calculating error and delta at each layer
        # Output
        self.output_error = y - output # error in this layer
        self.output_delta = self.output_error*self.sigmoidPrime(output) # apply derivative of sigmoid to error
        
        # L2
        self.L2_error = self.output_delta.dot(self.output_weights.T) 
        self.L2_delta = self.L2_error*self.sigmoidPrime(self.activated_L2)
        
        # L1
        self.L1_error = self.L2_delta.dot(self.L2_weights.T) 
        self.L1_delta = self.L1_error*self.sigmoidPrime(self.activated_L1)
        
        
        ## Update all weights
        self.L1_weights += X.T.dot(self.L1_delta) 
        self.L2_weights += self.activated_L1.T.dot(self.L2_delta)
        self.output_weights += self.activated_L2.T.dot(self.output_delta)
        
    def train (self, X, y):
        output = self.feed_forward(X)
        self.backward(X, y, output)

In [ ]:
model = NN_2L16()
epochs=1000
for i in range(epochs): 
    if i+1 in [1,2,3] or (i+1) % 500 == 0:
        print('+---------- EPOCH', i+1, '-----------+')
        #print("Input: \n", X) 
        #print("Actual Output: \n", y)  
        #print("Predicted Output: \n" + str(model.feed_forward(X))) 
        print("Loss: \n" + str(np.mean(np.square(y - model.feed_forward(X))))) # mean sum squared loss
        print("\n")
    model.train(X, y)

## 4) Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy. 

- Use the Heart Disease Dataset (binary classification)
- Use an appropriate loss function for a binary classification task
- Use an appropriate activation function on the final layer of your network. 
- Train your model using verbose output for ease of grading.
- Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
- When hyperparameter tuning, show you work by adding code cells for each new experiment. 
- Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
- You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

### Baseline : 1 Hidden 16 node Layer 

In [ ]:
%%time
# random seed to reproduce later
seed = 42
np.random.seed(seed)

def create_model():
    model = Sequential()
    model.add(Dense(16, input_dim=45, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# instantiate model obj
model = KerasClassifier(build_fn=create_model, verbose=0)

# GridSearchCV hyperparameters
batch_size = [20]
epochs = [20]
param_grid = dict(batch_size=batch_size, epochs=epochs)
c_v = StratifiedKFold(n_splits=4,
                      shuffle=True,
                      random_state=seed) # 4-fold CV

# Create Grid Search
cv_grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    n_jobs=-1, 
                    cv=c_v)

# Fit
cv_grid_result1 = cv_grid.fit(X, y)